# **套件匯入**

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import calendar
import seaborn as sns
import datetime as dt

# **理賠資料**
## 將兩張理賠表匯入

In [2]:
df1 = pd.read_excel("CLAIM_ACCT_FIN_1.xlsx")
df2 = pd.read_excel("CLAIM_ACCT_FIN_2.xlsx")

## 將兩張理賠表合併

In [3]:
df2.index = [120000+i for i in range(len(df2))]
frames = [df1,df2]
df = pd.concat(frames)

In [4]:
df

,INJURED_RK,Claim_RK,Policy_RK,BundleSubtype2,illness_code,illness_desc,DiagnosisCode_DESC,claim_settle_dt,REIMBURSED_YR_TW,INSURED_RK,POLICY_HOLDER_RK,MATURITY_BENEFICIARY_RK,DEATH_BENEFICIARY_RK
0,14,4034,72224,5.N疾病醫療,A09,感染性胃腸炎及大腸炎,01.傳染病和寄生蟲病,2017-03-10,8505.00,14,212635,NaN,NaN
1,14,62576,72224,4.C重大疾病,N19,腎衰竭,14.泌尿生殖系統疾病,2016-08-19,4200.00,14,212635,NaN,NaN
2,14,90970,72224,5.N疾病醫療,H26.9,白內障,07.眼和附器疾病,2016-12-04,29102.85,14,212635,NaN,NaN
3,14,230243,72224,1.D身故給付,V99,運輸意外事故,20.疾病和死亡的外因,2017-03-10,2131618.65,14,212635,NaN,NaN
4,24,157920,91033,5.N疾病醫療,R19.7,"腹瀉,未特定",18.症狀異常所見，不可歸類,2017-06-29,32452.35,30372,70614,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
234423,249733,30121,166605,6.N意外醫療,Y99.8,其他外因狀態,99.不知道不想猜,2015-07-09,33600.00,249733,250207,NaN,250207.0
234424,249733,86463,166605,5.N疾病醫療,D33,大腦及中樞神經系統其他部位之良性腫瘤,02.腫瘤,2016-11-02,231.00,249733,250207,NaN,250207.0
234425,249733,109718,166605,6.N意外醫療,Z04.2,來院接受對工作意外後之檢查及觀察,21.影響健康狀態與保健機構接觸的,2017-02-11,1163.40,249733,250207,NaN,250207.0
234426,249733,204841,166605,6.N意外醫療,V23,摩拖車騎士與汽車，小貨車或箱型車碰撞受傷,20.疾病和死亡的外因,2017-10-29,12478.20,249733,250207,NaN,250207.0


## 在新的理賠表中做運算，產生新的資料欄

In [5]:
df3 = df.set_index(['POLICY_HOLDER_RK','Policy_RK']).sort_values(['POLICY_HOLDER_RK','Policy_RK','claim_settle_dt'])
df3['計數'] = 1
df3['累積理賠金額'] = df3.groupby('Policy_RK')['REIMBURSED_YR_TW'].transform('cumsum')
df3['初次理賠時間'] = df3.groupby('Policy_RK')['claim_settle_dt'].transform(min)
df3['累積理賠次數'] = df3.groupby('Policy_RK')['計數'].transform('cumsum')
df3 = df3.drop(columns=['計數'])
df3['結案後120天'] = df3['claim_settle_dt'] + dt.timedelta(days=120)
df3['結案後180天'] = df3['claim_settle_dt'] + dt.timedelta(days=180)
df3['結案後360天'] = df3['claim_settle_dt'] + dt.timedelta(days=360)

## 將處理完的理賠檔存檔

In [6]:
df3.to_excel('ready_CLAIM_ACCT_FIN.xlsx')

In [213]:
# df3 = df3.loc[~df3.index.duplicated(keep='last')]
# df3 = df3.groupby(level=df3.index.names)  
# df3 = df3.last()
# df3

## 查看某位要保人的理賠資訊

In [7]:
df3.loc[130950].tail()

,INJURED_RK,Claim_RK,BundleSubtype2,illness_code,illness_desc,DiagnosisCode_DESC,claim_settle_dt,REIMBURSED_YR_TW,INSURED_RK,MATURITY_BENEFICIARY_RK,DEATH_BENEFICIARY_RK,累積理賠金額,初次理賠時間,累積理賠次數,結案後120天,結案後180天,結案後360天
Policy_RK,,,,,,,,,,,,,,,,,
105429,130950,160458,5.N疾病醫療,C57,其他及未明示女性生殖器官之惡性腫瘤,02.腫瘤,2017-07-05,16800.0,130950,NaN,NaN,202650.0,2015-09-04,21,2017-11-02,2018-01-01,2018-06-30
105429,130950,169555,5.N疾病醫療,C57,其他及未明示女性生殖器官之惡性腫瘤,02.腫瘤,2017-07-28,16800.0,130950,NaN,NaN,219450.0,2015-09-04,22,2017-11-25,2018-01-24,2018-07-23
105429,130950,178279,5.N疾病醫療,C57,其他及未明示女性生殖器官之惡性腫瘤,02.腫瘤,2017-08-19,23100.0,130950,NaN,NaN,242550.0,2015-09-04,23,2017-12-17,2018-02-15,2018-08-14
105429,130950,187467,5.N疾病醫療,C57,其他及未明示女性生殖器官之惡性腫瘤,02.腫瘤,2017-09-13,30450.0,130950,NaN,NaN,273000.0,2015-09-04,24,2018-01-11,2018-03-12,2018-09-08
105429,130950,222326,5.N疾病醫療,C57,其他及未明示女性生殖器官之惡性腫瘤,02.腫瘤,2017-12-14,18900.0,130950,NaN,NaN,291900.0,2015-09-04,25,2018-04-13,2018-06-12,2018-12-09


# **再購資料**

In [8]:
df4 = pd.read_excel("COV_ACCT_FIN.xlsx")

In [9]:
df4.sort_values(['POLICY_HOLDER_RK','Policy_RK','EFFECTIVE_DT'])

,INSURED_RK,Policy_RK,RIDER_CD,payment_period,EFFECTIVE_DT,SHORT_NAME,prod_detail2,POLICY_HOLDER_RK,AFYP_NT,MATURITY_BENEFICIARY_RK,DEATH_BENEFICIARY_RK
13965,26347,56714,0,月繳,2017-10-29,PAR,AHc,4,3969.00,26347.0,NaN
12107,23125,179172,1,月繳,2018-11-24,CAB,AHb,4,7207.20,NaN,NaN
12108,23125,179173,0,月繳,2018-11-24,NPBBR,AHc,4,1197.00,23125.0,NaN
12109,23125,179173,0,月繳,2018-11-24,NAI,AHc,4,705.60,23125.0,NaN
12110,23125,179173,0,月繳,2018-11-24,HIR,AHa,4,1965.60,23125.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
134471,251945,177625,1,躉繳保費,2018-08-31,BVA,ILP,251945,315000.00,NaN,120128.0
79401,134583,90488,0,年繳,2017-08-31,N1TR,REG,251951,1037.40,251951.0,251951.0
79402,134583,90488,1,年繳,2017-08-31,NPHI,AHa,251951,12852.00,251951.0,251951.0
79555,134869,90491,0,年繳,2017-09-08,N1TR,REG,251951,937.65,251951.0,251951.0


# **理賠再購合併**

## 要vs要 合併

In [10]:
df3['再購(120天)'] = 0 
for i in range(len(df3.index)):
    time_list = df4[df4['POLICY_HOLDER_RK']==df3.index[i][0]]['EFFECTIVE_DT'].unique()
    for time in time_list:
        if (df3.iloc[i,6] < time) and (time < df3.iloc[i,14]):
            df3.iloc[i,17] = 1
            break
    if i % 10000 == 0:
        print("第{}筆資料已標記!!".format(i))

第0筆資料已標記!!
第10000筆資料已標記!!
第20000筆資料已標記!!
第30000筆資料已標記!!
第40000筆資料已標記!!
第50000筆資料已標記!!
第60000筆資料已標記!!
第70000筆資料已標記!!
第80000筆資料已標記!!
第90000筆資料已標記!!
第100000筆資料已標記!!
第110000筆資料已標記!!
第120000筆資料已標記!!
第130000筆資料已標記!!
第140000筆資料已標記!!
第150000筆資料已標記!!
第160000筆資料已標記!!
第170000筆資料已標記!!
第180000筆資料已標記!!
第190000筆資料已標記!!
第200000筆資料已標記!!
第210000筆資料已標記!!
第220000筆資料已標記!!
第230000筆資料已標記!!


In [11]:
df3['再購(180天)'] = 0 
for i in range(len(df3.index)):
    time_list = df4[df4['POLICY_HOLDER_RK']==df3.index[i][0]]['EFFECTIVE_DT'].unique()
    for time in time_list:
        if (df3.iloc[i,6] < time) and (time < df3.iloc[i,15]):
            df3.iloc[i,18] = 1
            break
    if i % 10000 == 0:
        print("第{}筆資料已標記!!".format(i))

第0筆資料已標記!!
第10000筆資料已標記!!
第20000筆資料已標記!!
第30000筆資料已標記!!
第40000筆資料已標記!!
第50000筆資料已標記!!
第60000筆資料已標記!!
第70000筆資料已標記!!
第80000筆資料已標記!!
第90000筆資料已標記!!
第100000筆資料已標記!!
第110000筆資料已標記!!
第120000筆資料已標記!!
第130000筆資料已標記!!
第140000筆資料已標記!!
第150000筆資料已標記!!
第160000筆資料已標記!!
第170000筆資料已標記!!
第180000筆資料已標記!!
第190000筆資料已標記!!
第200000筆資料已標記!!
第210000筆資料已標記!!
第220000筆資料已標記!!
第230000筆資料已標記!!


In [12]:
df3['再購(360天)'] = 0 
for i in range(len(df3.index)):
    time_list = df4[df4['POLICY_HOLDER_RK']==df3.index[i][0]]['EFFECTIVE_DT'].unique()
    for time in time_list:
        if (df3.iloc[i,6] < time) and (time < df3.iloc[i,16]):
            df3.iloc[i,19] = 1
            break
    if i % 10000 == 0:
        print("第{}筆資料已標記!!".format(i))

第0筆資料已標記!!
第10000筆資料已標記!!
第20000筆資料已標記!!
第30000筆資料已標記!!
第40000筆資料已標記!!
第50000筆資料已標記!!
第60000筆資料已標記!!
第70000筆資料已標記!!
第80000筆資料已標記!!
第90000筆資料已標記!!
第100000筆資料已標記!!
第110000筆資料已標記!!
第120000筆資料已標記!!
第130000筆資料已標記!!
第140000筆資料已標記!!
第150000筆資料已標記!!
第160000筆資料已標記!!
第170000筆資料已標記!!
第180000筆資料已標記!!
第190000筆資料已標記!!
第200000筆資料已標記!!
第210000筆資料已標記!!
第220000筆資料已標記!!
第230000筆資料已標記!!


## 理賠再購合併檔案輸出

In [13]:
df3.to_excel('理賠再購合併.xlsx')

In [14]:
df3

INJURED_RK  Claim_RK BundleSubtype2 illness_code  \
POLICY_HOLDER_RK Policy_RK                                                     
3                49082            4324    104018        5.N疾病醫療          D36   
4                56714           26347     90482        6.N意外醫療          W18   
                 56714           26347    184848        6.N意外醫療          W18   
5                84973           93160    183589        5.N疾病醫療          D24   
6                65407           94922    175992        5.N疾病醫療       L03.90   
...                                ...       ...            ...          ...   
251945           102196         123213    112437        6.N意外醫療          V23   
251947           16872           86041    179452        6.N意外醫療        Y99.8   
251949           100554         161848    113170        6.N意外醫療          V99   
251951           96970          134869    174868        5.N疾病醫療          D36   
251954           74765          156320    136040        4.C重大疾病        I25.1   

                                    illness_desc DiagnosisCode_DESC  \
POLICY_HOLDER_RK Policy_RK                                            
3                49082              其他及未明示部位良性腫瘤              02.腫瘤   
4                56714             其他滑倒、摔倒或絆倒和跌落        20.疾病和死亡的外因   
                 56714             其他滑倒、摔倒或絆倒和跌落        20.疾病和死亡的外因   
5                84973                    乳房良性腫廇              02.腫瘤   
6                65407                     蜂窩組織炎       12.皮膚和皮下組織疾病   
...                                          ...                ...   
251945           102196     摩拖車騎士與汽車，小貨車或箱型車碰撞受傷        20.疾病和死亡的外因   
251947           16872                    其他外因狀態          99.不知道不想猜   
251949           100554                   運輸意外事故        20.疾病和死亡的外因   
251951           96970              其他及未明示部位良性腫瘤              02.腫瘤   
251954           74765            自體的冠狀動脈粥樣硬化心臟病          09.循環系統疾病   

                           claim_settle_dt  REIMBURSED_YR_TW  INSURED_RK  \
POLICY_HOLDER_RK Policy_RK                                                 
3                49082          2017-01-20           9189.60        4324   
4                56714          2016-12-01           1837.50       26347   
                 56714          2017-09-06           2857.05       26347   
5                84973          2017-09-02           2835.00       93160   
6                65407          2017-08-13           4200.00       94922   
...                                    ...               ...         ...   
251945           102196         2017-02-19            813.75      123213   
251947           16872          2017-08-23           1627.50       86041   
251949           100554         2017-02-24         262237.50      161848   
251951           96970          2017-08-11            546.00      134869   
251954           74765          2017-04-29          16825.20      251954   

                            MATURITY_BENEFICIARY_RK  DEATH_BENEFICIARY_RK  \
POLICY_HOLDER_RK Policy_RK                                                  
3                49082                          3.0                   NaN   
4                56714                      26347.0                   NaN   
                 56714                      26347.0                   NaN   
5                84973                      93160.0                   NaN   
6                65407                          NaN                   NaN   
...                                             ...                   ...   
251945           102196                    251945.0              251945.0   
251947           16872                          NaN                   NaN   
251949           100554                    251949.0               31420.0   
251951           96970                          NaN                   NaN   
251954           74765                          NaN                   NaN   

                               累積理賠金額     初次理賠時間  累積理賠次數    結案後120天  \
P

# **理賠再購檔.客戶屬性檔合併**

In [16]:
df4 = pd.read_excel('CUST_PROPERTY_FIN_1.xlsx')
df5 = pd.read_excel('CUST_PROPERTY_FIN_2.xlsx')

In [17]:
df5.index = [65000+i for i in range(len(df5))]
frames = [df4,df5]
df6 = pd.concat(frames)

In [18]:
df6

,CUST_RK,ternure_m,recency_m,SIN,SIN_his,REG,REG_his,ILP,ILP_his,AHa,...,WEALTH_LEVEL,CLIENT_MARITAL,CLIENT_INCOME,DIGI_FLG,TOPCARD,GENDER,stick_level2,cust_group2,TOTAL_AUM,INSURED_DOB
0,24,201,201,0,0,1,1,0,0,0,...,W7,NaN,0,0,0,1,S10,G4,396720.0,1984-04-04
1,152,175,175,0,0,1,1,0,0,1,...,W6,NaN,0,0,0,1,S10,G4,165870.0,1951-02-26
2,155,227,227,0,0,1,1,0,0,1,...,W7,NaN,0,0,0,1,S10,G4,NaN,1976-07-19
3,158,165,165,0,0,0,0,0,0,1,...,W7,NaN,0,0,0,1,S10,G4,68362.0,1978-12-09
4,204,181,181,0,0,1,1,0,0,1,...,W7,NaN,0,0,0,1,S10,G4,68400.0,1976-09-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130482,251944,271,4,1,1,1,1,0,0,1,...,W2,M,1900000,0,1,1,S04,G0,2885720.0,1966-10-06
130483,251945,255,54,0,0,1,1,0,0,1,...,W5,M,247000,0,0,1,S07,G4,300840.3,1966-01-09
130484,251947,135,102,0,0,1,1,0,0,1,...,W7,NaN,0,0,0,1,S09,G4,123291.0,1979-05-16
130485,251951,125,34,0,0,1,1,0,0,1,...,W5,M,665000,0,0,1,S07,G4,224832.7,1972-06-22


In [19]:
df7 = df3.merge(right=df6, how='left', left_on='POLICY_HOLDER_RK',right_on='CUST_RK')

In [21]:
df7.index= df3.index

In [22]:
df7

INJURED_RK  Claim_RK BundleSubtype2 illness_code  \
POLICY_HOLDER_RK Policy_RK                                                     
3                49082            4324    104018        5.N疾病醫療          D36   
4                56714           26347     90482        6.N意外醫療          W18   
                 56714           26347    184848        6.N意外醫療          W18   
5                84973           93160    183589        5.N疾病醫療          D24   
6                65407           94922    175992        5.N疾病醫療       L03.90   
...                                ...       ...            ...          ...   
251945           102196         123213    112437        6.N意外醫療          V23   
251947           16872           86041    179452        6.N意外醫療        Y99.8   
251949           100554         161848    113170        6.N意外醫療          V99   
251951           96970          134869    174868        5.N疾病醫療          D36   
251954           74765          156320    136040        4.C重大疾病        I25.1   

                                    illness_desc DiagnosisCode_DESC  \
POLICY_HOLDER_RK Policy_RK                                            
3                49082              其他及未明示部位良性腫瘤              02.腫瘤   
4                56714             其他滑倒、摔倒或絆倒和跌落        20.疾病和死亡的外因   
                 56714             其他滑倒、摔倒或絆倒和跌落        20.疾病和死亡的外因   
5                84973                    乳房良性腫廇              02.腫瘤   
6                65407                     蜂窩組織炎       12.皮膚和皮下組織疾病   
...                                          ...                ...   
251945           102196     摩拖車騎士與汽車，小貨車或箱型車碰撞受傷        20.疾病和死亡的外因   
251947           16872                    其他外因狀態          99.不知道不想猜   
251949           100554                   運輸意外事故        20.疾病和死亡的外因   
251951           96970              其他及未明示部位良性腫瘤              02.腫瘤   
251954           74765            自體的冠狀動脈粥樣硬化心臟病          09.循環系統疾病   

                           claim_settle_dt  REIMBURSED_YR_TW  INSURED_RK  \
POLICY_HOLDER_RK Policy_RK                                                 
3                49082          2017-01-20           9189.60        4324   
4                56714          2016-12-01           1837.50       26347   
                 56714          2017-09-06           2857.05       26347   
5                84973          2017-09-02           2835.00       93160   
6                65407          2017-08-13           4200.00       94922   
...                                    ...               ...         ...   
251945           102196         2017-02-19            813.75      123213   
251947           16872          2017-08-23           1627.50       86041   
251949           100554         2017-02-24         262237.50      161848   
251951           96970          2017-08-11            546.00      134869   
251954           74765          2017-04-29          16825.20      251954   

                            MATURITY_BENEFICIARY_RK  ...  WEALTH_LEVEL  \
POLICY_HOLDER_RK Policy_RK                           ...                 
3                49082                          3.0  ...           NaN   
4                56714                      26347.0  ...           NaN   
                 56714                      26347.0  ...           NaN   
5                84973                      93160.0  ...           NaN   
6                65407                          NaN  ...           NaN   
...                                             ...  ...           ...   
251945           102196                    251945.0  ...            W5   
251947           16872                          NaN  ...            W7   
251949           100554                    251949.0  ...           NaN   
251951           96970                          NaN  ...            W5   
251954           74765                          NaN  ...            W6   

                            CLIENT_MARITAL CLIENT_INCOME  DIGI_FLG TOPCARD  \
POLICY_HOLDER_RK Policy_RK       

In [23]:
df8 = df7[['INJURED_RK', 'Claim_RK', 'BundleSubtype2', 'illness_code',
       'illness_desc', 'DiagnosisCode_DESC', 'claim_settle_dt',
       'REIMBURSED_YR_TW', 'INSURED_RK', 'MATURITY_BENEFICIARY_RK',
       'DEATH_BENEFICIARY_RK', '累積理賠金額', '初次理賠時間', '累積理賠次數', '結案後120天',
       '結案後180天', '結案後360天', 'CUST_RK',
       'ternure_m', 'recency_m', 'SIN', 'SIN_his', 'REG', 'REG_his', 'ILP',
       'ILP_his', 'AHa', 'AHa_his', 'AHb', 'AHb_his', 'AHc', 'AHc_his', 'AHd',
       'AHd_his', 'VIP_CLASS', 'VIP', 'WEALTH_LEVEL', 'CLIENT_MARITAL',
       'CLIENT_INCOME', 'DIGI_FLG', 'TOPCARD', 'GENDER', 'stick_level2',
       'cust_group2', 'TOTAL_AUM', 'INSURED_DOB','再購(120天)', '再購(180天)', '再購(360天)']]

In [24]:
df8

INJURED_RK  Claim_RK BundleSubtype2 illness_code  \
POLICY_HOLDER_RK Policy_RK                                                     
3                49082            4324    104018        5.N疾病醫療          D36   
4                56714           26347     90482        6.N意外醫療          W18   
                 56714           26347    184848        6.N意外醫療          W18   
5                84973           93160    183589        5.N疾病醫療          D24   
6                65407           94922    175992        5.N疾病醫療       L03.90   
...                                ...       ...            ...          ...   
251945           102196         123213    112437        6.N意外醫療          V23   
251947           16872           86041    179452        6.N意外醫療        Y99.8   
251949           100554         161848    113170        6.N意外醫療          V99   
251951           96970          134869    174868        5.N疾病醫療          D36   
251954           74765          156320    136040        4.C重大疾病        I25.1   

                                    illness_desc DiagnosisCode_DESC  \
POLICY_HOLDER_RK Policy_RK                                            
3                49082              其他及未明示部位良性腫瘤              02.腫瘤   
4                56714             其他滑倒、摔倒或絆倒和跌落        20.疾病和死亡的外因   
                 56714             其他滑倒、摔倒或絆倒和跌落        20.疾病和死亡的外因   
5                84973                    乳房良性腫廇              02.腫瘤   
6                65407                     蜂窩組織炎       12.皮膚和皮下組織疾病   
...                                          ...                ...   
251945           102196     摩拖車騎士與汽車，小貨車或箱型車碰撞受傷        20.疾病和死亡的外因   
251947           16872                    其他外因狀態          99.不知道不想猜   
251949           100554                   運輸意外事故        20.疾病和死亡的外因   
251951           96970              其他及未明示部位良性腫瘤              02.腫瘤   
251954           74765            自體的冠狀動脈粥樣硬化心臟病          09.循環系統疾病   

                           claim_settle_dt  REIMBURSED_YR_TW  INSURED_RK  \
POLICY_HOLDER_RK Policy_RK                                                 
3                49082          2017-01-20           9189.60        4324   
4                56714          2016-12-01           1837.50       26347   
                 56714          2017-09-06           2857.05       26347   
5                84973          2017-09-02           2835.00       93160   
6                65407          2017-08-13           4200.00       94922   
...                                    ...               ...         ...   
251945           102196         2017-02-19            813.75      123213   
251947           16872          2017-08-23           1627.50       86041   
251949           100554         2017-02-24         262237.50      161848   
251951           96970          2017-08-11            546.00      134869   
251954           74765          2017-04-29          16825.20      251954   

                            MATURITY_BENEFICIARY_RK  ...  DIGI_FLG  TOPCARD  \
POLICY_HOLDER_RK Policy_RK                           ...                      
3                49082                          3.0  ...       NaN      NaN   
4                56714                      26347.0  ...       NaN      NaN   
                 56714                      26347.0  ...       NaN      NaN   
5                84973                      93160.0  ...       NaN      NaN   
6                65407                          NaN  ...       NaN      NaN   
...                                             ...  ...       ...      ...   
251945           102196                    251945.0  ...       0.0      0.0   
251947           16872                          NaN  ...       0.0      0.0   
251949           100554                    251949.0  ...       NaN      NaN   
251951           96970                          NaN  ...       0.0      0.0   
251954           74765                          NaN  ...       0.0      0.0   

                           GENDER  stick_level

In [25]:
df8.to_excel('理賠再購屬性合併.xlsx')